I want to de-alias all VAD files including the ones in motion. To do this I will import functions from Alex's VADS_2 file, which contains functions that perform all desried actions. -- Julia Buhrman 02/20/2024

In [1]:
#imports all needed modules

import pyart
import numpy as np
from datetime import datetime, timedelta
import glob
# from radarcalc import *
import matplotlib.pyplot as plt
import pandas as pd
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import gc
from astropy.convolution import convolve
from boto.s3.connection import S3Connection
import tempfile
import copy
import matplotlib
import xarray as xr
from datetime import datetime
# import from Alex's VAD_2 notebook which I changed to a .py file (so we can import stuff from it)
#from VADS_2_Copy import dealias_Ka, vehicle_correction_vad <== MAKES JUPYTER LAB BUFFER IDK WHY


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



I guess just copy and paste the functions from VADS_2 :(

In [2]:
import pyart
import numpy as np
from datetime import datetime, timedelta
import glob
# from radarcalc import *
import matplotlib.pyplot as plt
import pandas as pd
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import gc
from astropy.convolution import convolve
from boto.s3.connection import S3Connection
import tempfile
import copy
import matplotlib


def dealias_Ka(radar,PPIGC_flag=False):
    '''
    This function aims to take care of all the nitty gritty customizations of pyarts dealiasing specifically for TTUKa radar data.

    Parameter: radar (pyart object), PPIGC_flag (boolean *kwarg)
    Returns: radar (pyart object) with corrected_velocity field and velocity_texture field added on

    How it works:
    1. Calculates velocity texture and creates a histogram based on all of the magnitudes of textures at each bin for the entire volume
    2. loops through each indivual sweep
    3. infinite loop to iterate between the minimum amount of texture between textures 1 and 6 and the maximum amount of texture in the histogram, with 0.5 m/s steps
    4. Each iteration, create a gatefilter that filters the bins above that texture value and dealias it with the gatefilter to ignore the high textured regions
    5. If the maximum texture is reached before breaking, set the gatefilter to mask textures above 12 m/s
    6. if the scan is an RHI, run a 2 pass variance filter along each ray.
        This convolves a 71 sized boxcar with the data, and if the difference between the point the boxcar is centered on and the mean of the boxcar is greater than the nyquist, then add/subtract 2*nyquist to that point
    7. Then, take the difference between the mean of the bottom 4 rays of the dealiased velocity in the RHI and the bottom 4 rays of the aliased velocity
    8. If the absolute difference is larger than the nyquist, the either subtract 2* nyquist or add 2*nyquist to the entire sweep and break the loop
    9. If the absolute difference is less than the nyquist, no fixes need to be applied, break the loop
    10. If the scan is a PPI, do steps 8 and 9, but skip steps 6 and 7
    11. Outside the infinite loop, add the option of the PPIGC_flag, a boolean flag that will help maintain ground clutter in PPIs at 0 m/s
        This works by identifying regions of very low spectrum width (<0.1 m/s), and setting the velocity in those regions = 0. Please note, this may introduce artificial speckles of 0 in real data, where spectrum width is noisy
    12. Apply the alias fix algorithm which convolves a boxcar of specified boxcar of size 9 with the data, and if the variance between the middle pixel of interest and the mean is greater than the nyquist, then flip it back over
    12. At the end of each sweep, assign the data from that processed sweep into a dictionary, then add the dictionary as the corrected_velocity field.
    
    The aforementioned method is FAR from perfect, but is as robust as I can do currently. One thing to improve this though is to use the UNRAVEL algorithm: https://github.com/vlouf/dealias
    The UNRAVEL algorithm shows remarkable error characteristics compared to "competitors", possibly at a time cost, which isn't a HUGE deal for us. Downside is it may not work for our "volumes" since they are temporally uncorrelated and are not full volumes through the atmosphere
    '''
    
    vel_texture = pyart.retrieve.calculate_velocity_texture(radar, vel_field='velocity', wind_size=3)
    radar.add_field('velocity_texture', vel_texture, replace_existing=True)
    hist, bins = np.histogram(radar.fields['velocity_texture']['data'][~np.isnan(radar.fields['velocity_texture']['data'])], bins=150)
    bins = (bins[1:]+bins[:-1])/2.0
    gatefilter = pyart.filters.GateFilter(radar)
    velocity_dealiased = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=radar.instrument_parameters['nyquist_velocity']['data'][0], centered=True) #standin, data will be replaced

    for swp_id in range(radar.nsweeps):
        #get indices from beginning and ending of sweep
        sw_start = radar.sweep_start_ray_index['data'][swp_id]
        sw_end = radar.sweep_end_ray_index['data'][swp_id]+1

        counter = 0
        while True: #do an infinite loop and either break it when the data is unfolded correctly or when the max texture is reached
            #if the bin with the lowest count between textures 1 and 6 + i*0.5 is less than the maximum amount of bins
            if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                gatefilter.exclude_above('velocity_texture', bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5)
                nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]
                vede = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=nyq,
                                                                        centered=True, gatefilter=gatefilter)
            else:
                gatefilter.exclude_above('velocity_texture', 12)#bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]])
                nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]
                vede = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=nyq,
                                                                        centered=True, gatefilter=gatefilter)


            np.ma.set_fill_value(vede['data'], np.nan)
            #extract mask so we can apply the correct gatefilters on later
            mask=np.ma.getmask(vede['data'])

            #apply mask to velocity field and fix the small blips from dealiasing
            if radar.scan_type == 'rhi':
                #pass 1 of variance filtering along the ray.
                #Convolves a 71 sized boxcar with the data, and if the difference between the point the boxcar is centered on and the mean of the boxcar is greater than the nyquist, then add/subtract 2*nyquist to that point
                vel = vede['data'].filled()

                for sw in range(np.shape(vel)[0]):
                    mean = convolve(vel[sw,:],np.ones(71))
                    var = vel[sw,:]-mean
                    high_idx = var > nyq
                    low_idx = var < -nyq
                    vel[sw,:][high_idx] = vel[sw,:][high_idx] - 2*nyq
                    vel[sw,:][low_idx] = vel[sw,:][low_idx] + 2*nyq
                vede['data']=np.ma.masked_array(vel,mask=mask,fill_value=np.nan)

                #pass 2 of variance filtering along the ray. In case there are errant folds than need to be folded back
                vel = vede['data'].filled()
                for sw in range(np.shape(vel)[0]):
                    mean = convolve(vel[sw,:],np.ones(71))
                    var = vel[sw,:]-mean
                    high_idx = var > nyq
                    low_idx = var < -nyq
                    vel[sw,:][high_idx] = vel[sw,:][high_idx] - 2*nyq
                    vel[sw,:][low_idx] = vel[sw,:][low_idx] + 2*nyq
                vede['data']=np.ma.masked_array(vel,mask=mask,fill_value=np.nan)

                #find means of the bottom 4 rays of the RHI(should be close to 0) and compare the dealiased velocities to the aliased velocities
                np.ma.set_fill_value(radar.fields['velocity']['data'], np.nan)
                meanvelal = np.mean(radar.fields['velocity']['data'][sw_start:sw_start+4,:].filled()[~np.isnan(radar.fields['velocity']['data'][sw_start:sw_start+4,:].filled())])
                meanveldeal = np.mean(vede['data'][sw_start:sw_start+4,:].filled()[~np.isnan(vede['data'][sw_start:sw_start+4,:].filled())])
                if np.abs(meanvelal-meanveldeal) < nyq: #nyq is an arbitrary threshold and should be tuned
                    break
                if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                    if (meanvelal-meanveldeal) > 0:
                        vede['data'][sw_start:sw_end,:] += 2*nyq
                    else:
                        vede['data'][sw_start:sw_end,:] -= 2*nyq
                    break
            if radar.scan_type == 'ppi':
                np.ma.set_fill_value(radar.fields['velocity']['data'], np.nan)
                meanvelal = np.mean(radar.fields['velocity']['data'][sw_start:sw_end,:].filled()[~np.isnan(radar.fields['velocity']['data'][sw_start:sw_end,:].filled())])
                meanveldeal = np.mean(vede['data'][sw_start:sw_end,:].filled()[~np.isnan(vede['data'][sw_start:sw_end,:].filled())])
                if np.abs(meanvelal-meanveldeal) < nyq: #nyq is an arbitrary threshold and should be tuned
                    break
                if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                    if (meanvelal-meanveldeal) > 0:
                        vede['data'][sw_start:sw_end,:] += 2*nyq
                    else:
                        vede['data'][sw_start:sw_end,:] -= 2*nyq
                    break
            counter+=1
            
        #put alias fix inside here instead of calling it to make it more portable
        delta=3
        mean = convolve(vede['data'][sw_start:sw_end,:],np.ones((delta,delta))/delta**2.)
        mean[0,:] = vede['data'][sw_start:sw_end,:][0,:]
        mean[-1,:] = vede['data'][sw_start:sw_end,:][-1,:]
        var = vede['data'][sw_start:sw_end,:] - mean

        high_idx = np.logical_and(var > nyq, var < 4*nyq)
        low_idx = np.logical_and(var < -nyq, var > -4*nyq)

        vede['data'][sw_start:sw_end,:][high_idx] = vede['data'][sw_start:sw_end,:][high_idx] - 2*nyq
        vede['data'][sw_start:sw_end,:][low_idx] = vede['data'][sw_start:sw_end,:][low_idx] + 2*nyq

        #corrects ground clutter by arbitrarily setting the velocity equal to 0 where spectrum width is less than 0.075 m/s
        if PPIGC_flag == True:
            if radar.scan_type == 'ppi':
                sw = radar.fields['spectrum_width']['data'][sw_start:sw_end,:].filled()
                vel = radar.fields['velocity']['data'][sw_start:sw_end,:].filled()
                mask = sw<0.1
                vede['data'][sw_start:sw_end,:] = np.where(~mask,vede['data'][sw_start:sw_end,:],0)

        velocity_dealiased['data'][sw_start:sw_end,:] = vede['data'][sw_start:sw_end,:]
        velocity_dealiased['data'][sw_start:sw_end,:] = alias_fix(velocity_dealiased['data'][sw_start:sw_end,:],nyq,delta=9)
    radar.add_field('corrected_velocity', velocity_dealiased, replace_existing=True)

    return radar

def alias_fix(vel,nyq,delta=3):
    '''
    !!!!!!!!!!!!!!!!!!
    Removes dealiasing errors around the periphery of a folded region

    Parameters: velocity array (array), nyquist velocity (number), size of window (int, must be odd, unity is no change)
    Returns: cleaned velocity array (array)
    '''
    mean = convolve(vel,np.ones((delta,delta))/delta**2.)
    mean[0,:] = vel[0,:]
    mean[-1,:] = vel[-1,:]
    var = vel - mean

    high_idx = np.logical_and(var > nyq, var < 4*nyq)
    low_idx = np.logical_and(var < -nyq, var > -4*nyq)

    vel[high_idx] = vel[high_idx] - 2*nyq
    vel[low_idx] = vel[low_idx] + 2*nyq

    return vel

def get_radar_from_aws(site, datetime_t, datetime_te):
    """
    Get the closest volume of NEXRAD data to a particular datetime.
    Parameters
    ----------
    site : string
        four letter radar designation
    datetime_t : datetime
        desired date time
    Returns
    -------
    radar : Py-ART Radar Object
        Radar closest to the queried datetime
    """

    # First create the query string for the bucket knowing
    # how NOAA and AWS store the data
    my_pref = datetime_t.strftime('%Y/%m/%d/') + site

    # Connect to the bucket
    conn = S3Connection(anon = True)
    bucket = conn.get_bucket('noaa-nexrad-level2')

    # Get a list of files
    bucket_list = list(bucket.list(prefix = my_pref))

    # we are going to create a list of keys and datetimes to allow easy searching
    keys = []
    datetimes = []

    # populate the list
    for i in range(len(bucket_list)):
        this_str = str(bucket_list[i].key)
        if 'gz' in this_str:
            endme = this_str[-22:-4]
            fmt = '%Y%m%d_%H%M%S_V0'
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])

        if this_str[-3::] == 'V06':
            endme = this_str[-19::]
            fmt = '%Y%m%d_%H%M%S_V06'
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])

    # find the closest available radar to your datetime
    closest_datetime_b = _nearestDate(datetimes, datetime_t)
    closest_datetime_e = _nearestDate(datetimes, datetime_te)

    index_b = datetimes.index(closest_datetime_b)
    index_e = datetimes.index(closest_datetime_e)

    radar_namelist = keys[index_b:index_e+1]
    radar_list=[]
    for i in range(np.shape(radar_namelist)[0]):
        localfile = tempfile.NamedTemporaryFile()
        radar_namelist[i].get_contents_to_filename(localfile.name)
        radar_list.append(pyart.io.read(localfile.name))
    return radar_namelist,radar_list

def getLocation(lat1, lon1, brng, distancekm):
    lat1 = lat1 * np.pi / 180.0
    lon1 = lon1 * np.pi / 180.0
    #earth radius
    R = 6378.1
    #R = ~ 3959 MilesR = 3959
    bearing = (brng / 90.)* np.pi / 2.

    lat2 = np.arcsin(np.sin(lat1) * np.cos(distancekm/R) + np.cos(lat1) * np.sin(distancekm/R) * np.cos(bearing))
    lon2 = lon1 + np.arctan2(np.sin(bearing)*np.sin(distancekm/R)*np.cos(lat1),np.cos(distancekm/R)-np.sin(lat1)*np.sin(lat2))
    lon2 = 180.0 * lon2 / np.pi
    lat2 = 180.0 * lat2 / np.pi
    return lat2, lon2

def _nearestDate(dates, pivot):
    return min(dates, key=lambda x: abs(x - pivot))

def find_nearest(array, value):
    '''
    Function to find index of the array in which the value is closest to

    Parameters: array (array), value (number)
    Returns: index (int)

    Example: xind = CM1calc.find_nearest(x,5)
    '''

    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return idx


def vehicle_correction_vad(radar,df):
    '''
    Function that creates a 'vad_corrected_velocity' field that can be used for vad calculations, 
    but should be general enough to use for stationary VADs as well as moving PPIs. 
    Other than adding the new field, the radar times are smoothly interpolated and the azimuths are 
    corrected via the GPS pandas dataframe.
    
    Parameters: pyart radar object (object), pandas dataframe of appropriate radarGPS file (dataframe)
    Returns: pyart radar object (object), speed (float), speed variance (float), bearing (float), bearing variance (float), 
             latitude (float), latitude variance (float), longitude (float), longitude variance (float)
    
    Example: radar, velmean, velvar, bearmean, bearvar, latmean, latvar, lonmean, lonvar = vehicle_correction_vad(radar,df)

    p.s. only works if the velocity is already dealiased and there is a 'corrected_velocity' field
         also only works if a single sweep is extracted, example: radar = radar.extract_sweeps([0])
    '''
    
    #orders the time to increase monotonically instead of having a massive step jump in the middle
    roll_mag = (np.argmax(np.abs(np.gradient(radar.time['data'])))+1)
    times = np.roll(radar.time['data'],-roll_mag) 
    
    #a complicated way to create linear increasing times (instead of steps) that start at 0 seconds after the time datum and increase to the middle of the second max time plateau (if confused, plotting it is helpful)
    #from now on, we are going to assume ray_times is the fractional seconds after the time datum the ray is gathered, and we need to roll it back to match with the rest of the data
    ray_times = np.roll(np.arange(0,((np.unique(times)[-2])/(find_nearest(times,np.unique(times)[-2])+int(np.sum(radar.time['data']==np.unique(times)[-2])/2)))*len(times)+1e-11,((np.unique(times)[-2])/(find_nearest(times,np.unique(times)[-2])+int(np.sum(radar.time['data']==np.unique(times)[-2])/2)))),roll_mag)

    radar.time['data']=ray_times

    print([datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+[f'{h:06}' for h in df['hhmmss[UTC]'].astype(int)]][19660:19680])
    print(datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ'))
    
    df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+df['hhmmss[UTC]']]
    beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index
    endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.ceil(np.amax(ray_times))+1
    dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)]
    dfscan = dfscan.astype({'Bearing[degrees]': 'float'})
    dfscan = dfscan.astype({'Velocity[knots]': 'float'})

    ray_bearings = np.interp(ray_times,np.arange(len(dfscan)),dfscan['Bearing[degrees]'])
    ray_speeds = np.interp(ray_times,np.arange(len(dfscan)),dfscan['Velocity[knots]'])

    print('velocity [kts]',dfscan['Velocity[knots]'].mean(),'+-',dfscan['Velocity[knots]'].var())
    speed = dfscan['Velocity[knots]'].mean()
    print('bearing',dfscan['Bearing[degrees]'].mean(),'+-',dfscan['Bearing[degrees]'].var())
    bearing = dfscan['Bearing[degrees]'].mean()
    print('latitude',dfscan['Latitude'].astype(float).mean(),'+-',dfscan['Latitude'].astype(float).var())
    lat = dfscan['Latitude'].astype(float).mean()
    print('longitude',dfscan['Longitude'].astype(float).mean(),'+-',dfscan['Longitude'].astype(float).var())
    lon = dfscan['Longitude'].astype(float).mean()

    radar.azimuth['data'] += ray_bearings[:-1] #bearing

    rad_vel = copy.deepcopy(radar.fields['corrected_velocity'])

    rad_vel['data']+=(np.cos(np.deg2rad(radar.azimuth['data']-ray_bearings[:-1]))*(ray_speeds[:-1]/1.94384)*np.cos(np.deg2rad(radar.fixed_angle['data'][0])))[:,np.newaxis]

    #fix mask, remove points very close to radar as well as the very last bin, more often than not, = bad data
    rad_vel['data'].mask[:,:5] = True
    rad_vel['data'].mask[:,-1] = True
    radar.add_field('vad_corrected_velocity', rad_vel, replace_existing=True)

    return radar, dfscan['Velocity[knots]'].mean(),dfscan['Velocity[knots]'].var(),dfscan['Bearing[degrees]'].mean(),dfscan['Bearing[degrees]'].var(),dfscan['Latitude'].astype(float).mean(),dfscan['Latitude'].astype(float).var(),dfscan['Longitude'].astype(float).mean(),dfscan['Longitude'].astype(float).var()


In [3]:
# bring in the files
# gps data for first ka
ka1gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka1_20220523.txt')
# scan data for first ka
ka1scan = pd.read_csv('/Users/juliabman/Desktop/research2024/20220523_Ka1_scan_log.txt', dtype=str)

# dealiased vad data created from VADS.ipynb for ka 1, but not bearing corrected
vad_fileska1 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka1/*'))

In [4]:
dealiased_data = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/*.nc'))

In [5]:
np.size(vad_fileska1)

344

In [6]:
ka1gps

,ddmmyy,hhmmss[UTC],Longitude,Latitude,Velocity[knots],Bearing[degrees]
0,230522,183124,-102.031783,33.588547,0.00,140.92
1,230522,183125,-102.031783,33.588547,0.00,140.92
2,230522,183126,-102.031783,33.588547,0.00,140.92
3,230522,183127,-102.031783,33.588547,0.00,140.92
4,230522,183128,-102.031783,33.588547,0.00,140.92
...,...,...,...,...,...,...
31901,240522,32353,-102.032235,33.589625,0.22,52.29
31902,240522,32354,-102.032227,33.589630,0.50,52.29
31903,240522,32355,-102.032210,33.589645,1.53,35.15
31904,240522,32356,-102.032200,33.589662,3.10,17.84


In [10]:
# data for ka 2

ka2gps = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka2_20220523.txt')
# scan data for first ka
ka2scan = pd.read_csv('/Users/juliabman/Desktop/research2024/20220523_Ka2_scan_log.txt', dtype=str)                    

# dealiased vad data created from VADS.ipynb for ka 2, but not bearing corrected
vad_fileska2 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka2/*'))
vad_fileska2_only_raw = sorted(glob.glob('/Users/juliabman/Desktop/research2024/julia_product_raw_ka2/*.RAW*'))

In [11]:
np.size(vad_fileska2_only_raw)

325

In [12]:
np.size(vad_fileska2)

410

In [13]:
# convert Datetime to pandas so it will be useable later
ka1scan['Datetime'] = pd.to_datetime(ka1scan['Datetime'])
ka2scan['Datetime'] = pd.to_datetime(ka2scan['Datetime']) 

In [14]:
# convert ddmmyy to string since thats what the code requires

ka1gps['ddmmyy'] = ka1gps['ddmmyy'].astype(str)
ka1gps['hhmmss[UTC]'] = ka1gps['hhmmss[UTC]'].astype(str)

ka2gps['ddmmyy'] = ka2gps['ddmmyy'].astype(str)
ka2gps['hhmmss[UTC]'] = ka2gps['hhmmss[UTC]'].astype(str)


In [58]:
ka2gps

,ddmmyy,hhmmss[UTC],Longitude,Latitude,Velocity[knots],Bearing[degrees]
0,230522,200239,-102.026500,33.581025,39.61,178.73
1,230522,200240,-102.026495,33.580842,39.42,178.93
2,230522,200241,-102.026492,33.580660,39.24,179.41
3,230522,200242,-102.026490,33.580478,39.08,179.40
4,230522,200243,-102.026488,33.580298,38.87,179.35
...,...,...,...,...,...,...
22101,240522,25355,-102.032913,33.592712,14.29,358.40
22102,240522,25356,-102.032918,33.592775,12.85,351.77
22103,240522,25357,-102.032937,33.592828,11.35,336.85
22104,240522,25358,-102.032970,33.592868,10.18,317.92


In [15]:
#ka1gps.datetime

In [16]:
ka1_one = vad_fileska1[0]

In [17]:
ka1_one[53:-8]

'/Ka1220523201819'

In [18]:
ka2_one = vad_fileska2[0]

In [19]:
ka2_one[54:-8]

'Ka2220523205016'

Try to get the data from 05/24 into a separate array to force it through the vehicle correction function:

In [20]:
datetime.strptime('20220524', '%Y%m%d')

datetime.datetime(2022, 5, 24, 0, 0)

In [21]:
#np.where(ka1gps.datetime == datetime.strptime('20220524', '%Y%m%d'))

In [22]:
#np.where(ka1gps.datetime == np.datetime64(datetime(2022,5,24)))

In [23]:
# start_time = np.datetime64(datetime(2022,5,24,))
# end_time = np.datetime64(datetime(2022,5,25))
# times_0524 = ka1gps[(ka1gps.datetime >= start_time) & (ka1gps.datetime < end_time)]

In [24]:
#start_time

In [25]:
ka1gps_0524_old = ka1gps.loc[19668:]

Reset the index numbers so that 19668 becomes 0, hopefully that fixes the function freaking out.

In [26]:
ka1gps_0524 = ka1gps_0524_old.reset_index(drop = True)

In [27]:
ka1gps_0524

,ddmmyy,hhmmss[UTC],Longitude,Latitude,Velocity[knots],Bearing[degrees]
0,240522,0,-102.639515,33.771422,47.08,220.83
1,240522,1,-102.639678,33.771248,47.39,216.80
2,240522,2,-102.639827,33.771065,47.77,213.45
3,240522,3,-102.639968,33.770873,48.13,211.76
4,240522,4,-102.640105,33.770678,48.62,210.97
...,...,...,...,...,...,...
12233,240522,32353,-102.032235,33.589625,0.22,52.29
12234,240522,32354,-102.032227,33.589630,0.50,52.29
12235,240522,32355,-102.032210,33.589645,1.53,35.15
12236,240522,32356,-102.032200,33.589662,3.10,17.84


In [28]:
ka1gps_0524['ddmmyy'] = ka1gps_0524['ddmmyy'].astype(str)
ka1gps_0524['hhmmss[UTC]'] = ka1gps_0524['hhmmss[UTC]'].astype(str)

In [29]:
times_0524 = ka1gps_0524
times_0524

,ddmmyy,hhmmss[UTC],Longitude,Latitude,Velocity[knots],Bearing[degrees]
0,240522,0,-102.639515,33.771422,47.08,220.83
1,240522,1,-102.639678,33.771248,47.39,216.80
2,240522,2,-102.639827,33.771065,47.77,213.45
3,240522,3,-102.639968,33.770873,48.13,211.76
4,240522,4,-102.640105,33.770678,48.62,210.97
...,...,...,...,...,...,...
12233,240522,32353,-102.032235,33.589625,0.22,52.29
12234,240522,32354,-102.032227,33.589630,0.50,52.29
12235,240522,32355,-102.032210,33.589645,1.53,35.15
12236,240522,32356,-102.032200,33.589662,3.10,17.84


Use Alex's function to dealias all VADs, including the ones in motion, and download them (radar objects) to netcdf so that I don't have to run this function every time:

In [30]:
'''

RUNNING THIS CODE WILL DEALIAS ALL DATA AND WILL TAKE HOURS!!! ALL NETCDFS HAVE ALREADY BEEN SAVED AT THE PATH

path = '/Users/juliabman/Desktop/dealiased_data/'
de_aliased_vads_ka1 = []
for the_file in vad_fileska1:
    radar = pyart.io.read(the_file)
    print(the_file)
    dealiasing = dealias_Ka(radar, PPIGC_flag=False)
    de_aliased_vads_ka1.append(dealiasing)
    #file_name = the_file.split('/')
    new_file_name = pyart.io.write_cfradial(path+the_file[54:-8]+"_dealiased.nc", radar)

'''

'\n\nRUNNING THIS CODE WILL DEALIAS ALL DATA AND WILL TAKE HOURS!!! ALL NETCDFS HAVE ALREADY BEEN SAVED AT THE PATH\n\npath = \'/Users/juliabman/Desktop/dealiased_data/\'\nde_aliased_vads_ka1 = []\nfor the_file in vad_fileska1:\n    radar = pyart.io.read(the_file)\n    print(the_file)\n    dealiasing = dealias_Ka(radar, PPIGC_flag=False)\n    de_aliased_vads_ka1.append(dealiasing)\n    #file_name = the_file.split(\'/\')\n    new_file_name = pyart.io.write_cfradial(path+the_file[54:-8]+"_dealiased.nc", radar)\n\n'

In [31]:
test = pyart.io.read(vad_fileska2[100])

/Users/juliabman/miniforge3/envs/Research/lib/python3.9/site-packages/pyart/io/sigmet.py:146: UserWarning: Unknown type: 17, returning raw data
  sigmet_data, sigmet_metadata = sigmetfile.read_data(full_xhdr=full_xhdr)
/Users/juliabman/miniforge3/envs/Research/lib/python3.9/site-packages/pyart/io/sigmet.py:146: RuntimeWarning: invalid value encountered in sqrt
  sigmet_data, sigmet_metadata = sigmetfile.read_data(full_xhdr=full_xhdr)


In [32]:
test.fields

{'total_power': {'units': 'dBZ',
  'standard_name': 'equivalent_reflectivity_factor',
  'long_name': 'Total power',
  'coordinates': 'elevation azimuth range',
  'data': masked_array(
    data=[[25.5, 18.5, -30.5, ..., --, --, -7.0],
          [25.5, 18.5, -31.5, ..., --, --, -7.0],
          [25.5, 18.5, -31.5, ..., --, --, -7.0],
          ...,
          [25.5, 18.5, -24.5, ..., --, --, -7.0],
          [25.5, 18.5, -24.5, ..., --, --, -7.0],
          [25.5, 18.5, -25.0, ..., --, --, -7.0]],
    mask=[[False, False, False, ...,  True,  True, False],
          [False, False, False, ...,  True,  True, False],
          [False, False, False, ...,  True,  True, False],
          ...,
          [False, False, False, ...,  True,  True, False],
          [False, False, False, ...,  True,  True, False],
          [False, False, False, ...,  True,  True, False]],
    fill_value=1e+20,
    dtype=float32),
  '_FillValue': -9999.0},
 'reflectivity': {'units': 'dBZ',
  'standard_name': 'equivale

In [33]:
test_my_copy = pyart.io.read(vad_fileska2[300])
test_my_copy.fields

{'total_power': {'units': 'dBZ',
  'standard_name': 'equivalent_reflectivity_factor',
  'long_name': 'Total power',
  'coordinates': 'elevation azimuth range',
  'data': masked_array(
    data=[[25.5, 18.5, -24.0, ..., --, --, 7.0],
          [25.5, 19.0, -25.0, ..., --, --, 7.0],
          [25.5, 18.5, -25.0, ..., --, --, 7.0],
          ...,
          [25.5, 18.5, -21.0, ..., --, --, 7.0],
          [25.5, 18.5, -21.0, ..., --, --, 7.0],
          [25.5, 18.5, -21.0, ..., --, --, 7.0]],
    mask=[[False, False, False, ...,  True,  True, False],
          [False, False, False, ...,  True,  True, False],
          [False, False, False, ...,  True,  True, False],
          ...,
          [False, False, False, ...,  True,  True, False],
          [False, False, False, ...,  True,  True, False],
          [False, False, False, ...,  True,  True, False]],
    fill_value=1e+20,
    dtype=float32),
  '_FillValue': -9999.0},
 'reflectivity': {'units': 'dBZ',
  'standard_name': 'equivalent_ref

In [34]:
'''

# RUNNING THIS CODE WILL DEALIAS ALL DATA AND WILL TAKE HOURS!!! ALL NETCDFS HAVE ALREADY BEEN SAVED AT THE PATH

path = '/Users/juliabman/Desktop/dealiased_data/ka2/'
de_aliased_vads_ka2 = []
for the_file in vad_fileska2[276:]: # I have 196 and it errored out so let's see if itll continue
    radar = pyart.io.read(the_file)
    print(the_file)
    dealiasing = dealias_Ka(radar, PPIGC_flag=False)
    de_aliased_vads_ka2.append(dealiasing)
    #file_name = the_file.split('/')
    new_file_name = pyart.io.write_cfradial(path+the_file[54:-8]+"_dealiased.nc", radar)
    #print(new_file_name)
'''

'\n\n# RUNNING THIS CODE WILL DEALIAS ALL DATA AND WILL TAKE HOURS!!! ALL NETCDFS HAVE ALREADY BEEN SAVED AT THE PATH\n\npath = \'/Users/juliabman/Desktop/dealiased_data/ka2/\'\nde_aliased_vads_ka2 = []\nfor the_file in vad_fileska2[276:]: # I have 196 and it errored out so let\'s see if itll continue\n    radar = pyart.io.read(the_file)\n    print(the_file)\n    dealiasing = dealias_Ka(radar, PPIGC_flag=False)\n    de_aliased_vads_ka2.append(dealiasing)\n    #file_name = the_file.split(\'/\')\n    new_file_name = pyart.io.write_cfradial(path+the_file[54:-8]+"_dealiased.nc", radar)\n    #print(new_file_name)\n'

In [36]:
#np.size(de_aliased_vads_ka1)

Bring in all the netcdf files for ka1 that can be converted into radar objects:

In [49]:
dealiased_data_ka1 = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/ka1/*.nc'))

In [50]:
vad_fileska1 = sorted(glob.glob('/Users/juliabman/Desktop/research2024/product_raw_ka1/*.nc'))

In [51]:
dealiased_data_ka2 = sorted(glob.glob('/Users/juliabman/Desktop/dealiased_data/ka2/*.nc'))

In [52]:
np.size(dealiased_data_ka1)

307

**There were 325 RAW files for Ka2, but 318 got dealiased. 7 were corrupted in various ways (missing velocity fields, etc.) **

In [41]:
np.size(dealiased_data)

307

In [42]:
np.size(vad_fileska1)

37

In [43]:
one = vad_fileska1[0]

In [44]:
one[-18:-3]

'Ka1220523205632'

344 total vad files exist, but only 307 are raw data were produced by the function. The other 37 have already been created by Alex's original vad code and were dealiased netcdfs and not raw data, so the true total is 307.

In [45]:
#all_dealiased_data_ka1

In [46]:
# path = '/Users/juliabman/Desktop/dealiased_data/'
# all_rest_of_ka1 = []
# for thefile in vad_fileska1[:]:
#     print(thefile)
#     radar = pyart.io.read(thefile)
#     print(radar)
#     write = pyart.io.write_cfradial(path+thefile[-18:-3]+"_dealiased.nc", radar)
#     print(write)
#     all_rest_of_ka1.append(write)

In [ ]:
time_check = []
                        
for the_file in all_dealiased_data_ka1[:10]:
    read = pyart.io.read_cfradial(the_file) #converts netcdfs back to radar objects!
    print(the_file)
    times = read.time['data']
    time_check.append(times)

In [ ]:
# you can indeed see that they are being read in as radar objects
read

In [ ]:
first_one = pyart.io.read_cfradial(all_dealiased_data_ka1[0])

In [ ]:
#first_one.fields

The time field for these vad netcdfs are in seconds since scan started, which is not very useful. It's better to just pull the time from the title of the VAD.

Correct the vads for vehicle motion (can only do one day at a time??):

In [57]:
corrected_vehicle_vads_0523 = []
corrected_vads_file_names_0523 = []
for uncorrected_vad in dealiased_data_ka1:
    print(uncorrected_vad)
    read = pyart.io.read_cfradial(uncorrected_vad)
    correction_function = vehicle_correction_vad(read, ka1gps)
    corrected_vehicle_vads_0523.append(correction_function)
    corrected_vads_file_names_0523.append(uncorrected_vad)

/Users/juliabman/Desktop/dealiased_data/ka1/Ka1220523201819_dealiased.nc
[datetime.datetime(2022, 5, 23, 23, 59, 52), datetime.datetime(2022, 5, 23, 23, 59, 53), datetime.datetime(2022, 5, 23, 23, 59, 54), datetime.datetime(2022, 5, 23, 23, 59, 55), datetime.datetime(2022, 5, 23, 23, 59, 56), datetime.datetime(2022, 5, 23, 23, 59, 57), datetime.datetime(2022, 5, 23, 23, 59, 58), datetime.datetime(2022, 5, 23, 23, 59, 59), datetime.datetime(2022, 5, 24, 0, 0), datetime.datetime(2022, 5, 24, 0, 0, 1), datetime.datetime(2022, 5, 24, 0, 0, 2), datetime.datetime(2022, 5, 24, 0, 0, 3), datetime.datetime(2022, 5, 24, 0, 0, 4), datetime.datetime(2022, 5, 24, 0, 0, 5), datetime.datetime(2022, 5, 24, 0, 0, 6), datetime.datetime(2022, 5, 24, 0, 0, 7), datetime.datetime(2022, 5, 24, 0, 0, 8), datetime.datetime(2022, 5, 24, 0, 0, 9), datetime.datetime(2022, 5, 24, 0, 0, 10), datetime.datetime(2022, 5, 24, 0, 0, 11)]
2022-05-23 20:18:19
velocity [kts] 41.32212765957447 +- 13.54162580943571
bearing 2

KeyboardInterrupt: 

The function above (vehicle_correction_vad) for some reason only creates vads from 05/23 and not any from 05/24..?

In [54]:
np.size(corrected_vads_file_names_0523)

0

In [55]:
np.size(corrected_vehicle_vads_0523)

0

In [ ]:
1692/9 # 9 categories per list for each radar object

Make an array of times for 05/24 so we can force the function to run through the second day:

In [ ]:
dealiased_data_ka1_0524 = all_dealiased_data_ka1[188:]

In [ ]:
dealiased_data_ka1_0524

In [ ]:
np.size(dealiased_data_ka1_0524)

In [ ]:
ka1gps[19668:]

Attempting to get it to work through forcing it the dates:

In [ ]:
corrected_vehicle_vads_0524 = []
corrected_vads_file_names_0524 = []
for uncorrected_vad in all_dealiased_data_ka1[188:]:
    print(uncorrected_vad)
    read = pyart.io.read_cfradial(uncorrected_vad)
    correction_function = vehicle_correction_vad(read, ka1gps_0524)
    corrected_vehicle_vads_0524.append(correction_function)
    corrected_vads_file_names_0524.append(uncorrected_vad)

In [ ]:
read

In [ ]:
times_0524

In [ ]:
first_vad = corrected_vehicle_vads_0523[0]

In [ ]:
first_vad

In [ ]:
first_vad_radar = first_vad[0] # this shows the corrected_vehicle_vads are lists of objects

In [ ]:
first_vad_radar.time

Find where in the list the lat and lon are kept:

In [ ]:
first_vad_latitude = first_vad[5]

In [ ]:
first_vad_latitude

In [ ]:
first_vad_longitude = first_vad[7]

In [ ]:
first_vad_longitude

In [ ]:
ka1_corrected_vad_lats = []
ka1_corrected_vad_lons = []
for file in corrected_vehicle_vads:
    lat = file[5]
    lon = file[7]
    ka1_corrected_vad_lats.append(lat)
    ka1_corrected_vad_lons.append(lon)

In [ ]:
np.size(ka1_corrected_vad_lats)

Grab the times from the vad netcdf file names:

In [ ]:
vad_ka1_times = []
for time_grab in range(len(dealiased_data_ka1)):
    file = dealiased_data_ka1[time_grab]
    time_yoink = file[43:-13]
    time_yoink_datetime = datetime.strptime(time_yoink, '%y%m%d%H%M%S')
    vad_ka1_times.append(time_yoink_datetime)

In [ ]:
np.size(vad_ka1_times)

Grab the times from the CORRECTED dealiased vad files:

In [ ]:
vad_ka1_times_corrected_0523 = []
for time_grab in range(len(corrected_vads_file_names)):
    file = corrected_vads_file_names[time_grab]
    time_yoink = file[43:-13]
    time_yoink_datetime = datetime.strptime(time_yoink, '%y%m%d%H%M%S')
    vad_ka1_times_corrected_0523.append(time_yoink_datetime)

In [ ]:
np.size(vad_ka1_times_corrected)

In [ ]:
vad_ka1_times_corrected

In [ ]:
vad_ka1_times_corrected_0524 = []
for time_grab in range(len(corrected_vads_file_names)):
    file = corrected_vads_file_names[time_grab]
    time_yoink = file[43:-13]
    time_yoink_datetime = datetime.strptime(time_yoink, '%y%m%d%H%M%S')
    vad_ka1_times_corrected_0523.append(time_yoink_datetime)

In [ ]:
tobac_file = '/Users/juliabman/Desktop/research2024/tobac_Save/Track.nc'
tobac_features_xr = xr.open_dataset(tobac_file)

In [ ]:
tobac_features_xr

In [ ]:
tobac_times = tobac_features_xr['timestr']

In [ ]:
corrected_vehicle_vads_ka2 = []
corrected_vads_file_names_ka2 = []
for uncorrected_vad in dealiased_data_ka2:
    print(uncorrected_vad)
    read = pyart.io.read_cfradial(uncorrected_vad)
    correction_function = vehicle_correction_vad(read, ka2gps)
    corrected_vehicle_vads_ka2.append(correction_function)
    corrected_vads_file_names_ka2.append(uncorrected_vad)